In [30]:
import os
import xml.etree.ElementTree as ET
from PIL import Image
import shutil

In [31]:

# Dossiers de source
train_dir = 'train_zip/train'  # Dossier avec les images d'entraînement
test_dir = 'test_zip/test'    # Dossier avec les images de test

# Dossiers de destination
train_dest_dir = 'yolo_dataset/images/train'
test_dest_dir = 'yolo_dataset/images/test'

# Créer les répertoires de destination
os.makedirs(train_dest_dir, exist_ok=True)
os.makedirs(test_dest_dir, exist_ok=True)

# Copier les images dans les bons dossiers
for filename in os.listdir(train_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        shutil.copy(os.path.join(train_dir, filename), os.path.join(train_dest_dir, filename))

for filename in os.listdir(test_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        shutil.copy(os.path.join(test_dir, filename), os.path.join(test_dest_dir, filename))

print("Les images ont été copié dans 'yolo_dataset/images/train' et 'yolo_dataset/images/test'.")


Les images ont été copié dans 'yolo_dataset/images/train' et 'yolo_dataset/images/test'.


In [32]:
# Dossiers des fichiers XML et images 
xml_train_dir = 'yolo_dataset/labels/train'  # Dossier contenant les fichiers XML des annotations pour l'entraînement
xml_test_dir = 'yolo_dataset/labels/test'    # Dossier contenant les fichiers XML des annotations pour le test
image_train_dir = 'yolo_dataset/images/train'  # Dossier des images correspondantes pour l'entraînement
image_test_dir = 'yolo_dataset/images/test'    # Dossier des images correspondantes pour le test

os.makedirs(xml_test_dir, exist_ok=True)

# Copier les fichiers XML
for filename in os.listdir(test_dir):
    if filename.endswith('.xml'):
        shutil.copy(os.path.join(test_dir, filename), os.path.join(xml_test_dir, filename))

print("Les fichiers XML ont été copiés dans 'yolo_dataset/labels/test'.")
# Dossiers de sortie pour les labels YOLO
output_train_dir = 'yolo_labels/train'
output_test_dir = 'yolo_labels/test'

# Créer les répertoires de sortie si nécessaire
os.makedirs(output_train_dir, exist_ok=True)
os.makedirs(output_test_dir, exist_ok=True)

# Dictionnaire pour les mappages de classe (à adapter selon vos classes)
class_mapping = {
    'apple': 0,
    'banana': 1,
    'orange': 2,
    #'mixed': 3
}

# Fonction pour normaliser les coordonnées de la boîte englobante
def normalize_bbox(xmin, ymin, xmax, ymax, width, height):
    if width == 0 or height == 0:
        raise ValueError(f"Dimensions invalides : largeur={width}, hauteur={height}")
    
    x_center = (xmin + xmax) / 2.0 / width
    y_center = (ymin + ymax) / 2.0 / height
    bbox_width = (xmax - xmin) / width
    bbox_height = (ymax - ymin) / height
    return x_center, y_center, bbox_width, bbox_height

# Fonction pour convertir les fichiers XML en labels YOLO
def convert_xml_to_yolo(xml_file, image_dir, output_dir):
    xml_path = os.path.join(xml_test_dir, xml_file)
    image_path = os.path.join(image_dir, xml_file.replace('.xml', '.jpg'))
    
    print(f"Traitement du fichier XML : {xml_file}")

    # Charger l'arbre XML
    tree = ET.parse(xml_path)
    root = tree.getroot()

    # Récupérer la taille de l'image depuis le fichier XML
    size = root.find('size')
    width, height = 0, 0

    if size is not None:
        width = int(size.find('width').text) if size.find('width') is not None else 0
        height = int(size.find('height').text) if size.find('height') is not None else 0

    if width == 0 or height == 0:
        # Essayer de récupérer la taille de l'image depuis le fichier image
        if os.path.exists(image_path):
            with Image.open(image_path) as img:
                width, height = img.size
            print(f"Dimensions extraites de l'image {image_path}: largeur={width}, hauteur={height}")
        else:
            print(f"⚠️ L'image correspondante est manquante pour {xml_file}")
            return  # Si l'image est manquante, passer au fichier suivant

    # Vérifier si les dimensions sont valides
    if width == 0 or height == 0:
        print(f"⚠️ Dimensions invalides ou manquantes pour {xml_file}")
        return  # Passer au fichier suivant

    # Créer un fichier .txt pour chaque image
    txt_filename = os.path.splitext(xml_file)[0] + '.txt'
    with open(os.path.join(output_dir, txt_filename), 'w') as txt_file:
        # Parcourir les objets dans le fichier XML
        for obj in root.findall('object'):
            class_name = obj.find('name').text
            if class_name in class_mapping:
                class_id = class_mapping[class_name]
                bndbox = obj.find('bndbox')
                xmin = int(bndbox.find('xmin').text)
                ymin = int(bndbox.find('ymin').text)
                xmax = int(bndbox.find('xmax').text)
                ymax = int(bndbox.find('ymax').text)

                # Normaliser les coordonnées et écrire dans le fichier .txt
                try:
                    x_center, y_center, bbox_width, bbox_height = normalize_bbox(xmin, ymin, xmax, ymax, width, height)
                    txt_file.write(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n")
                except ValueError as e:
                    print(f"⚠️ Erreur de normalisation pour {xml_file}: {e}")
                    continue  # Passer à l'objet suivant
    
    # Copier l'image correspondante dans le bon répertoire de sortie
    try:
        shutil.copy(image_path, os.path.join(output_dir, xml_file.replace('.xml', '.jpg')))
    except Exception as e:
        print(f"⚠️ Impossible de copier l'image {image_path}: {e}")

# Fonction pour traiter tous les fichiers XML
def process_xml_files(xml_dir, image_dir, output_dir):
    for xml_file in os.listdir(xml_dir):
        if xml_file.endswith('.xml'):
            convert_xml_to_yolo(xml_file, image_dir, output_dir)

# Traiter les fichiers d'entraînement et de test
print("✅ Traitement des fichiers d'entraînement...")
process_xml_files(xml_train_dir, image_train_dir, output_train_dir)

print("✅ Traitement des fichiers de test...")
process_xml_files(xml_test_dir, image_test_dir, output_test_dir)

print("✅ Conversion terminée ! Les fichiers YOLO sont dans les dossiers 'yolo_labels/labels/train' et 'yolo_labels/labels/test'.")

Les fichiers XML ont été copiés dans 'yolo_dataset/labels/test'.
✅ Traitement des fichiers d'entraînement...
✅ Traitement des fichiers de test...
Traitement du fichier XML : apple_77.xml
Traitement du fichier XML : apple_78.xml
Traitement du fichier XML : apple_79.xml
Dimensions extraites de l'image yolo_dataset/images/test\apple_79.jpg: largeur=720, hauteur=720
Traitement du fichier XML : apple_80.xml
Traitement du fichier XML : apple_81.xml
Traitement du fichier XML : apple_82.xml
Traitement du fichier XML : apple_83.xml
Traitement du fichier XML : apple_84.xml
Traitement du fichier XML : apple_85.xml
Traitement du fichier XML : apple_86.xml
Traitement du fichier XML : apple_87.xml
Traitement du fichier XML : apple_88.xml
Traitement du fichier XML : apple_89.xml
Traitement du fichier XML : apple_90.xml
Traitement du fichier XML : apple_91.xml
Traitement du fichier XML : apple_92.xml
Dimensions extraites de l'image yolo_dataset/images/test\apple_92.jpg: largeur=600, hauteur=600
Traite

In [33]:
%pip install yolov5

Note: you may need to restart the kernel to use updated packages.
